In [2]:
import sys
sys.path.insert(0,"/home2/tathagato/summarization/MACSUM/naacl")
import os
from eval import evaluate
import pickle as pkl
import tqdm
import re


[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


loading brown clusters...
done; # words:  247339
loading word embeddings...
done; # words:  268810


In [3]:
attributes = ['extractiveness', 'specificity']
arguments = [
 '/scratch/tathagato/naacl/zero_shot/llama_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/zero_shot/mistral_extractiveness_and_specificity.pkl',
 
 '/scratch/tathagato/naacl/joint_mult_attribute_sft/llama_extractiveness_and_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/joint_mult_attribute_sft/mistral_extractiveness_and_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 
 '/scratch/tathagato/naacl/joint_mult_attribute_dpo/llama_extractiveness_and_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/joint_mult_attribute_dpo/mistral_extractiveness_and_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/llama_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/mistral_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/mistral_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_sft/llama_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',


 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/llama_specificity_then_extractiveness/model_specificity_then_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/mistral_extractiveness_then_specificity/model_extractiveness_then_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/mistral_specificity_then_extractiveness/model_specificity_then_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_sft/llama_extractiveness_then_specificity/model_extractiveness_then_specificity_results_extractiveness_and_specificity.pkl',
 
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/llama_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/mistral_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/mistral_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_multi_adapter_dpo/llama_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',


 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/llama_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/mistral_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/mistral_specificity_then_extractiveness/model_specificity_and_extractiveness_results_specificity_and_extractiveness.pkl',
 '/scratch/tathagato/naacl/multi_attribute_single_adapter_continued_dpo/llama_extractiveness_then_specificity/model_extractiveness_and_specificity_results_extractiveness_and_specificity.pkl'

]

print(len(arguments))









    







22


In [10]:

def extract_floats(string):
    # This pattern matches numbers with optional negative sign,
    # decimal point, and exponent notation
    pattern = r'-?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?'
    
    # Find all matches in the string
    matches = re.findall(pattern, string)
    
    # Convert matches to float
    floats = [float(match) for match in matches]
    
    return floats
def extract_words_around_then(filepath):
    # Check if "then" is in the filepath
    if "then" not in filepath:
        return ""
    
    # Regular expression to match the pattern
    pattern = r'_([^_]+)_then_([^_]+)_'
    
    # Search for the pattern in the filepath
    match = re.search(pattern, filepath)
    
    if match:
        # Extract the words before and after "then"
        word_before = match.group(1)
        word_after = match.group(2)
        return f"{word_before[0]}->{word_after[0]}"

def extract_meta_information(path):
    config = ""
    model = "Llama-3.1-Storm-8B"
    if "mistral" in path:
        model = "Mistral-7b-v3"
    method = "not sure"
    if "zero_shot" in path:
        method = "zero shot"

    if "adapter_fusion" in path:
        method = "adapter fusion"
        w1, w2 = extract_floats(path)[1], extract_floats(path)[2]
        config = f"{w1}, {w2}"

    elif "joint_mult_attribute" in path:
        method = "joint training"
    elif "multi_attribute_single_adapter_continued" in path:
        method = "single adapter continous"
        config = extract_words_around_then(path)
    elif "multi_attribute_multi_adapter" in path:
        method = "multi adapter"
        config = extract_words_around_then(path)
    
    objective = ""
    if "sft" in path:
        objective = "SFT"
    elif "dpo" in path:
        objective = "DPO"
    return model, method, objective, config

def display_specificity_and_extractiveness(res):
    extractive_keys = ['normal', 'high', 'fully']
    extractive_metrics = ['fragment_density', 'coverage', 'overlap' , 'f_value']
    specificity_keys = ['normal', 'high']
    specificity_metrics = ['specificity']
    specificity_res = res['result']['specificity']
    extractive_res = res['result']['extractiveness']
    extractive_outputs = []
    specificity_outputs = []
    model = res['model']
    method = res['method']
    objective = res['objective']
    
    # Collect metrics
    for key in extractive_keys:
        key_outputs = []
        for met in extractive_metrics:
            key_outputs.append(round(extractive_res[key][met], 2))
        extractive_outputs.append(key_outputs)
            
    for key in specificity_keys:
        key_outputs = []
        for met in specificity_metrics:
            key_outputs.append(round(specificity_res[key][met], 2))
        specificity_outputs.append(key_outputs)

    #print(extractive_outputs)
    #print(length_outputs)
    # Print in LaTeX format
    # print(f"\\multirow{{4}}{{*}}{{{method}}} & \\multirow{{4}}{{*}}{{{objective}}} & \\multirow{{4}}{{*}}{{}} & Length & {length_outputs[0][0]} & {length_outputs[1][0]} & {length_outputs[2][0]} & Density & {extractive_outputs[0][0]} & {extractive_outputs[1][0]} & {extractive_outputs[2][0]} \\\\")
    # print(f"& & & Compression Ratio & {length_outputs[0][1]} & {length_outputs[1][1]} & {length_outputs[2][1]} & Coverage & {extractive_outputs
    # print(f"& & & & & & & Overlap & {extractive_outputs[0][2]} & {extractive_outputs[1][2]} & {extractive_outputs[2][2]} \\\\")
    # print(f"& & & & & & & Rouge-P & {extractive_outputs[0][3]} & {extractive_outputs[1][3]} & {extractive_outputs[2][3]} \\\\")
    # print("\\hline")

    print(f"{method} & {objective} & {config} & {specificity_outputs[0][0]}/{specificity_outputs[1][0]}  & {extractive_outputs[0][0]}/{extractive_outputs[1][0]}/{extractive_outputs[2][0]} & {extractive_outputs[0][1]}/{extractive_outputs[1][1]}/{extractive_outputs[2][1]} & {extractive_outputs[0][3]} / {extractive_outputs[1][3]} / {extractive_outputs[2][3]} \\\\")


    


In [5]:
final_res = {}
for file in tqdm.tqdm(arguments):
    model , method, objective, config = extract_meta_information(file)
    try:
        res = evaluate(file, model=model, attributes=attributes)
        final_res[file] = {"result": res, "model": model, "method": method, "objective": objective, "config": config}
    except Exception as e:
        print(f"Error evaluating {file}: {e}")
        res = None


  0%|          | 0/22 [00:00<?, ?it/s]

evaluating Llama-3.1-Storm-8B with attributes ['extractiveness', 'specificity']
length of data is  547
/scratch/tathagato/naacl/zero_shot/llama_extractiveness_and_specificity.pkl is being evaluated on extractiveness


/home2/tathagato/miniconda3/envs/roy/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


extractiveness evaluation
--------------overall----------------
fragment_density : 3.200199879680703
gold_fragment_density : 3.5740066422781958
coverage : 0.9014329439259938
gold_coverage : 0.8769379470940913
overlap : 0.5050742784939468
gold_overlap : 0.4790941100760585
f_value : 0.054878413290794716
gold_f_value : 0.3935447733737069
rouge1 : 0.30272027718421596
rouge2 : 0.09527703970804517
rouge3 : 0.03977233701654131
rougeL : 0.18855604933777156
number : 547
-------------------------------------------------
--------------normal----------------
fragment_density : 3.0099243481493443
gold_fragment_density : 3.0470808464199073
coverage : 0.8988141482026891
gold_coverage : 0.866959576381
overlap : 0.4974804823954432
gold_overlap : 0.44346948972372485
f_value : 0.0540392147342753
gold_f_value : 0.3864833426448362
rouge1 : 0.3144294155971922
rouge2 : 0.09818711851902291
rouge3 : 0.04062122505840623
rougeL : 0.19388224916228083
number : 467
-------------------------------------------------


  5%|▍         | 1/22 [00:52<18:28, 52.79s/it]

...

size of specificity_scores:  547


specificity evaluation
--------------overall----------------
specificity : 0.494611784113485
gold_specificity : 0.5665536802926033
rouge1 : 0.2565839492036823
rouge2 : 0.0914175713482988
rouge3 : 0.042032769232167345
rougeL : 0.16639817060667422
number : 547
-------------------------------------------------
--------------normal----------------
specificity : 0.49423167276133056
gold_specificity : 0.5652978313432888
rouge1 : 0.26593422999599714
rouge2 : 0.09388839210899584
rouge3 : 0.04187736048738321
rougeL : 0.17079463855214208
number : 475
-------------------------------------------------
--------------high----------------
specificity : 0.497119463172838
gold_specificity : 0.5748387948887751
rouge1 : 0.19489806897660555
rouge2 : 0.07511701771870014
rouge3 : 0.043058035256784914
rougeL : 0.13739369457754594
number : 72
-------------------------------------------------
------------------------------------------------------------------------------------

 68%|██████▊   | 15/22 [13:03<06:01, 51.71s/it]

size of specificity_scores:  547


specificity evaluation
--------------overall----------------
specificity : 0.5617040540308044
gold_specificity : 0.5665536802926033
rouge1 : 0.30327042065115695
rouge2 : 0.09455666893956499
rouge3 : 0.03687011510549999
rougeL : 0.19109171707718836
number : 547
-------------------------------------------------
--------------normal----------------
specificity : 0.5623392824723551
gold_specificity : 0.5655243624969859
rouge1 : 0.31353840234669345
rouge2 : 0.0986090490560688
rouge3 : 0.038573640340418366
rougeL : 0.19694391562608451
number : 475
-------------------------------------------------
--------------high----------------
specificity : 0.557513310840019
gold_specificity : 0.5733443185275792
rouge1 : 0.23553026363199236
rouge2 : 0.06782221678207459
rouge3 : 0.025631580569580228
rougeL : 0.15248346276155372
number : 72
-------------------------------------------------
-----------------------------------------------------------------------------------

 73%|███████▎  | 16/22 [14:02<05:23, 53.97s/it]

size of specificity_scores:  547


specificity evaluation
--------------overall----------------
specificity : 0.7832405745048905
gold_specificity : 0.5665536802926032
rouge1 : 0.2231008400590433
rouge2 : 0.07692282420564298
rouge3 : 0.03231024341376778
rougeL : 0.13573394203407757
number : 547
-------------------------------------------------
--------------normal----------------
specificity : 0.7820514019654516
gold_specificity : 0.567765132645271
rouge1 : 0.23084636236372771
rouge2 : 0.07979011789502
rouge3 : 0.03324021039200728
rougeL : 0.13899098694695144
number : 475
-------------------------------------------------
--------------high----------------
specificity : 0.7910858100081342
gold_specificity : 0.558561459910421
rouge1 : 0.17200190818786112
rouge2 : 0.05800665056044732
rouge3 : 0.02617504459899331
rougeL : 0.11424649295609018
number : 72
-------------------------------------------------
----------------------------------------------------------------------------------------


In [11]:
for file,res in final_res.items():
    model , method, objective, config = extract_meta_information(file)
    res['model'] = model
    res['method'] = method
    res['objective'] = objective
    res['config'] = config
    #filters 
    if model == "Llama-3.1-Storm-8B":
        # print(file)
        # print(model)
        # print(method)
        # print(objective)
        # print("-------------")
        display_specificity_and_extractiveness(res)


zero shot &  &  & 0.54/0.51  & 3.01/3.43/5.33 & 0.9/0.92/0.92 & 0.05 / 0.05 / 0.07 \\


In [8]:
for file,res in final_res.items():
    model , method, objective, config = extract_meta_information(file)
    res['model'] = model
    res['method'] = method
    res['objective'] = objective
    res['config'] = config
    #filters 
    if model == "Mistral-7b-v3":
        # print(file)
        # print(model)
        # print(method)
        # print(objective)
        # print("-------------")
        display_length_and_extractiveness(res)